# Layer 5 of the cortical microcircuit model

**Modeling networks of spiking neurons using NEST**

**OCNC summer school 2018, 28.6.2018**

**Espen Hagen - espen.hagen@fys.uio.no**

This example implements a two-population network of layer 5 excitatory and inhibitory cells, and is based on the network description of:

[1] Potjans, T. C., & Diesmann, M. (2014). The cell-type specific cortical microcircuit: relating structure and activity in a full-scale spiking network model. Cerebral Cortex 24(3):785-806

Note that the full network model of [1] is part of the standard NEST release. 

<img src="files/PD2014.png">

In [ ]:
%pylab inline
pylab.rcParams.update({'figure.figsize': [10.0, 6.0]})

First, import NEST and all necessary modules for simulation, analysis and plotting.

In [ ]:
import nest
import numpy as np
import nest.raster_plot

## Parameters

The following parameters are extracted from [1], Table 5. Only layer 5 with its excitatory and inhibitory population will be simulated here. We also ignore inputs from other network populations than within layer 5.

**TODO** Fill in the missing parameters.

In [ ]:
# simulation parameters
T = 500.                   # simulation time (ms)
dt = 0.1                   # simulation resolution (ms)

# network parameters
N_L5E = 4850               # number of neurons in L5E
N_L5I = 1065               # number of neurons in L5I
    
K_L5E_L5E = 2038173        # number of synapses from L5E onto L5E 
K_L5E_L5I = 319602         # number of synapses from L5E onto L5I
K_L5I_L5I = 430775         # number of synapses from L5I onto L5I
K_L5I_L5E = 2411184        # number of synapses from L5I onto L5E

k_L5E_ext = 2000           # indegree of excitatory neurons to external poisson drive
k_L5I_ext = 1900           # indegree of inhibitory neurons to external poisson drive

# neuron parameters for model iaf_psc_exp
neuron_params = {
    'C_m'       : 250.,    # membrane capacity (pF)
    'I_e'       : 0.,      # external input current (pA)
    'tau_m'     : 10.,     # membrane time constant (ms)
    't_ref'     : 2.,      # absolute refractory period (ms)
    'tau_syn_ex': 0.5,     # excitatory postsynaptic current time constant (ms)
    'tau_syn_in': 0.5,     # inhibitory postsynaptic current time constant (ms)
    'V_reset'   : -65.,    # reset potential (mV)
    'E_L'       : -65.,    # resting potential (mV)
    'V_th'      : -50.     # spike threshold (mV)
}

# synapse parameters
J = 87.8                   # mean excitatory weight (pA)
sigma_J = 8.8              # standard deviation of excitatory weight (pA)
g = -4.                    # relative inhibitory weight 

d_E = 1.5                  # mean spike transmission delay for excitatory presynaptic neurons (ms)
sigma_d_E = 0.75           # standard deviation 
d_I = 0.8                  # mean spike transmission delay for inhibitory presynaptic neurons (ms)
sigma_d_I = 0.4            # standard deviation
 
# input parameters
bg_rate = 8.               # external poisson rate (Hz)
perturbation = False       # if True, insert external noise for a finite amount of time. See point 4*)


# 1) Create and connect neurons

**TODO** Reset the simulation kernel for avoiding interferences with previous NEST simulations.

In [ ]:
nest.ResetKernel()

Set simulation time resolution and number of threads (most computers have more than 1 CPU core):

In [ ]:
nest.SetKernelStatus({'resolution': dt, 'local_num_threads' : 4})

**TODO** Create the excitatory and inhibitory neuron populations with neurons of type **`iaf_psc_exp`**, the correct population sizes, and the given neuron parameters.

In [ ]:
pop_L5E = nest.Create('iaf_psc_exp', N_L5E)
nest.SetStatus(pop_L5E, params=neuron_params)

pop_L5I = nest.Create('iaf_psc_exp', N_L5I, params=neuron_params)

The excitatory connections (connections from L5E) are established.

**TODO** Specify the connection dictionary **`conn_dict`** for connections to L5E and to L5I. Which connection rule must be used (see documentation of **`Connect`**)?

In [ ]:
# excitatory connections (connections from L5E)

# specifying the synapse parameters
weight_dict = {
    'distribution': 'normal_clipped',
    'mu': J,
    'sigma': sigma_J,
    'low': 0.0
}
delay_dict = {
    'distribution': 'normal_clipped',
    'mu': d_E,
    'sigma': sigma_d_E,
    'low' : 0.1
}
syn_dict = {
    'weight': weight_dict,
    'delay': delay_dict
}

# connections to L5E
# specifying the connection parameters
conn_dict = {'rule': 'fixed_total_number', 'N': K_L5E_L5E}
nest.Connect(pop_L5E, pop_L5E, conn_dict, syn_dict)

# connections to L5I
conn_dict = {'rule': 'fixed_total_number', 'N': K_L5E_L5I}
nest.Connect(pop_L5E, pop_L5I, conn_dict, syn_dict)

**TODO** Create the inhibitory connections (connections from population L5I) in the same way as the excitatory ones. Pay attention where parameters must be adjusted.

In [ ]:
# specifying the synapse parameters
weight_dict = {
    'distribution': 'normal_clipped',
    'mu': J*g,
    'sigma': np.abs(sigma_J*g),
    'high': 0.0
}
delay_dict = {
    'distribution': 'normal_clipped',
    'mu': d_I,
    'sigma': sigma_d_I,
    'low' : 0.1
}
syn_dict = {
    'weight': weight_dict,
    'delay': delay_dict
}

# connections to L5E
# specifying the connection parameters
conn_dict = {'rule': 'fixed_total_number', 'N': K_L5I_L5E}
nest.Connect(pop_L5I, pop_L5E, conn_dict, syn_dict)

# connections to L5I
conn_dict = {'rule': 'fixed_total_number', 'N': K_L5I_L5I}
nest.Connect(pop_L5I, pop_L5I, conn_dict, syn_dict)


# 2) Create and connect devices

Poisson generators simulate neuron firing with the statistics of Poisson processes. Here, they emulate external excitatory input to the network.

**TODO** Create two poisson generators and connect them to the respective populations.
The given external rate **`bg_rate`** corresponds to the rate communicated by one synapse. How must the key **`rate`** of the poisson generator's **`params`** dictionary be set? The connection rule is **`all_to_all`**. For weights and delays, just take the the mean values given for excitatory neurons.

In [ ]:
# connections to L5E
poisson_generator_L5E = nest.Create('poisson_generator', 
                                    params={'rate': bg_rate * k_L5E_ext})
nest.Connect(poisson_generator_L5E, pop_L5E, 
             conn_spec='all_to_all', 
             syn_spec={'weight': J, 'delay': dt})

# connections to L5I
poisson_generator_L5I = nest.Create('poisson_generator', 
                                    params={'rate': bg_rate * k_L5I_ext})
nest.Connect(poisson_generator_L5I, pop_L5I, 
             conn_spec='all_to_all', 
             syn_spec={'weight': J, 'delay': dt})

**TODO** Set up and connect two spike detectors, one for each population.

In [ ]:
sd_L5E = nest.Create('spike_detector')
nest.Connect(pop_L5E, sd_L5E, 'all_to_all')

sd_L5I = nest.Create('spike_detector')
nest.Connect(pop_L5I, sd_L5I, 'all_to_all')

# 4*) Perturbation

**TODO (ADDITIONAL)** If **`perturbation == True`** in the section with the parameter definitions above, the network shall experience a perturbation for a certain time interval during the simulation. In our case, the perturbation consists of an additional poisson generator which connects to **`N_pert`** excitatory parrot neurons (Find the right model!) which then connect to L5E. Parrot neurons just repeat incoming spikes and, hence, can be used to generate correlated input.

In [ ]:
if perturbation:
    
    # parameters
    N_pert = 1000
    Nsyn_L5E_pert = 500000
    start = 200.
    stop = 300.
    pert_rate = 8.
    
    poisson_generator_stimulus = nest.Create('poisson_generator', 
                                             params={'rate': pert_rate, 
                                                     'start': start, 
                                                     'stop': stop})
    parrot_neurons = nest.Create('parrot_neuron', N_pert)
    nest.Connect(poisson_generator_stimulus, parrot_neurons, 'all_to_all')
    
    weight_dict = {
        'distribution': 'normal_clipped',
        'mu': J,
        'sigma': sigma_J,
        'low': 0.0
    }
    delay_dict = {
        'distribution': 'normal_clipped',
        'mu': d_E,
        'sigma': sigma_d_E,
        'low' : 0.1
    }
    syn_dict = {'weight': weight_dict, 'delay': delay_dict}
    
    conn_dict = {'rule': 'fixed_total_number', 'N': Nsyn_L5E_pert}
    nest.Connect(parrot_neurons, pop_L5E, conn_dict, syn_dict) 
    
    

# 3) Run simulation and analysis

**TODO** Run the simulation!

In [ ]:
nest.Simulate(T)

After the simulation, the spikes which were recorded with the spike detectors are read out. From each spike event, one can extract the spike time and the id of the sending neuron.

**TODO** Read out the data for L5I in the same way as for L5E.

In [ ]:
spike_senders_L5E = nest.GetStatus(sd_L5E)[0]['events']['senders']
spike_times_L5E = nest.GetStatus(sd_L5E)[0]['events']['times']
spike_senders_L5I = nest.GetStatus(sd_L5I)[0]['events']['senders']
spike_times_L5I = nest.GetStatus(sd_L5I)[0]['events']['times']

Finally, we create a raster plot to visualize the spiking activity of all neurons during the simulation ...

**TODO** Just uncomment the following lines.

In [ ]:
fig, ax = plt.subplots()
ax.plot(spike_times_L5E, spike_senders_L5E, 'r.', markersize=1, label='L5E')
ax.plot(spike_times_L5I, spike_senders_L5I, 'b.', markersize=1, label='L5I')
pylab.legend(loc=1)
ax.set_xlim(0,T)
ax.set_ylim(pop_L5E[0], pop_L5I[-1])
ax.set_xlabel('time (ms)')
ax.set_ylabel('neuron id')

... and we compute the average firing rate for each population.

**TODO** Determine the average rate per neuron by reading out the number of spikes registered by a spike detector during the simulation time. Pay attention to get the correct time units.

In [ ]:
rate_E = float(nest.GetStatus(sd_L5E)[0]['n_events']) / T * 1e3 / N_L5E
print "\nFiring rate E = %.1f spikes/s" % (rate_E)

rate_I = float(nest.GetStatus(sd_L5I)[0]['n_events']) / T * 1e3 / N_L5I
print "Firing rate I = %.1f spikes/s" % (rate_I)

Besides, NEST also provides build-in plotting tools for creating raster plots.

**TODO** Just uncomment the following lines.

In [ ]:
nest.raster_plot.from_device(sd_L5E, hist=True)
nest.raster_plot.from_device(sd_L5I, hist=True)

Look at source code of NEST's plotting functions:

In [ ]:
nest.raster_plot.from_device??

In [ ]:
nest.raster_plot._make_plot??